## Rudics Prawler Parser -> CSV for erddap

In [11]:
import os
import glob #python >= 3.5

import datetime

import pandas as pd

import seawater as sw
import calc.aanderaa_corrO2_sal as optode_O2_corr

In [12]:
#list id's in download path
root_path = '/Users/bell/ecoraid/2019/Moorings/19bsitaepr2a/raw/rudics/eclipse.pmel.noaa.gov/rudics/FOCI/M002'

id_dir = os.listdir(root_path)
id_dir = sorted(id_dir)


In [13]:
profile_counter = 972 #0,147,184,426,697,972 : 4:2, 4:3, 5:1, 5:2, 5:3, 6:1
month='5'
day='3'

for indfile in sorted(glob.glob(root_path+'/CM002_0['+month+']_['+day+']*')):
    print(indfile)
    with open(indfile) as f:
        for line in f:
            if ('%%' in line):
                flag = line.split()[0]
                if ('praw002' in line) or ('praw03' in line):
                    profile_counter += 1
                next(f)
                continue
            if not (line.strip()):
                continue
            if 'eng' in flag:
                with open("eng.csv", "a") as myfile:
                    myfile.write(line.strip()+'\n')
            if 'GPS' in flag:
                with open("gps.csv", "a") as myfile:
                    myfile.write(line.strip()+'\n')
            if 'BARO' in flag:
                with open("baro.csv", "a") as myfile:
                    myfile.write(line.strip()+'\n')
            if 'ATRH' in flag:
                with open("atrh.csv", "a") as myfile:
                    myfile.write(line.strip()+'\n')
            if 'WIND' in flag:
                with open("wind.csv", "a") as myfile:
                    myfile.write(line.strip()+'\n')
            if 'praw002' in flag:
                with open("praw002.csv", "a") as myfile:
                    myfile.write(str(profile_counter) + ','+line.strip()+'\n')        
            if 'praw03' in flag:
                with open("praw03.csv", "a") as myfile:
                    myfile.write(str(profile_counter) + ','+line.strip()+'\n')  
                    
print("Total Profile number:{profile_counter}".format(profile_counter=profile_counter))

/Users/bell/ecoraid/2019/Moorings/19bsitaepr2a/raw/rudics/eclipse.pmel.noaa.gov/rudics/FOCI/M002/CM002_05_30_2019
/Users/bell/ecoraid/2019/Moorings/19bsitaepr2a/raw/rudics/eclipse.pmel.noaa.gov/rudics/FOCI/M002/CM002_05_31_2019
Total Profile number:1022


In [14]:
### apply corrections to raw prawler data, merge met data
# a gps line gets put into the atrh file?

atrh = pd.read_csv('atrh.csv',header=None,names=['datetime','RH','AT'],
                   parse_dates=True,index_col='datetime')

wind = pd.read_csv('wind.csv',header=None,names=['datetime','U','USD','V','VSD'],
                   parse_dates=True,index_col='datetime')

baro = pd.read_csv('baro.csv',header=None,names=['datetime','BP'],
                   parse_dates=True,index_col='datetime')

In [15]:
#concat and save
metdata = pd.concat([baro, wind, atrh], axis=1, sort=False)
metdata['id']='19BSITAEPR2A'
metdata['dep_lat']=56.873
metdata['dep_lon']=-164.065

## simple qc
metdata=metdata[(metdata['AT']!='U')&(metdata['AT']!='D')]

In [16]:
"""p03 = pd.read_csv('praw03.csv',header=None,
                  names=['profilenumber','datetime','CD','CT','CC','OT','02','CH','TB'],
                   parse_dates=True,index_col='datetime')

## simple qc
p03=p03[(p03.index>='2000') & (p03.index<=datetime.datetime.now().strftime('%Y-%m-%d'))]
p03.index = pd.to_datetime(p03.index)

"""

"p03 = pd.read_csv('praw03.csv',header=None,\n                  names=['profilenumber','datetime','CD','CT','CC','OT','02','CH','TB'],\n                   parse_dates=True,index_col='datetime')\n\n## simple qc\np03=p03[(p03.index>='2000') & (p03.index<=datetime.datetime.now().strftime('%Y-%m-%d'))]\np03.index = pd.to_datetime(p03.index)\n\n"

In [17]:
p02 = pd.read_csv('praw002.csv',header=None,
                  names=['profilenumber','datetime','CD','CT','CC','OT','02','CH','TB'],
                   parse_dates=True,index_col='datetime')

## simple qc
p02=p02[(p02.index>='2000') & (p02.index<='2030')]
p02.index = pd.to_datetime(p02.index)

In [18]:
"""#initial deployment of instrument
p03.CD = p03.CD/100
p03.CT = p03.CT/1000
p03.CC = p03.CC/1000
p03.OT = p03.OT/1000
p03['02'] = p03['02']/100
### blocked sensor
p03.CH = (p03.CH-50)*0.0
p03.TB = (p03.TB-50)*0.0
# convert cond to salinity
p03['S'] = sw.salt(r=p03.CC/42.914,t=p03.CT,p=p03.CD)

### convert cond to salinity
p03.S = sw.salt(r=p03.CC/42.914,t=p03.CT,p=p03.CD)
### sal correct and convert oxygen
p03['O2_corr'] = optode_O2_corr.O2_sal_comp(oxygen_conc=p03['02'],
                                     salinity=p03.S,
                                     temperature=p03.CT)
p03['O2_corr_umkg'] = optode_O2_corr.O2_molar2umkg(oxygen_conc=p03.O2_corr,
                                     salinity=p03.S,
                                     temperature=p03.CT,
                                     pressure=p03.CD)

p03['O2PercentSat'] = optode_O2_corr.O2PercentSat(oxygen_conc=p03.O2_corr_umkg,
                                     salinity=p03.S,
                                     temperature=p03.CT,
                                     pressure=p03.CD)

p03['dep_lat']=56.873
p03['dep_lon']=-164.065
                        """

"#initial deployment of instrument\np03.CD = p03.CD/100\np03.CT = p03.CT/1000\np03.CC = p03.CC/1000\np03.OT = p03.OT/1000\np03['02'] = p03['02']/100\n### blocked sensor\np03.CH = (p03.CH-50)*0.0\np03.TB = (p03.TB-50)*0.0\n# convert cond to salinity\np03['S'] = sw.salt(r=p03.CC/42.914,t=p03.CT,p=p03.CD)\n\n### convert cond to salinity\np03.S = sw.salt(r=p03.CC/42.914,t=p03.CT,p=p03.CD)\n### sal correct and convert oxygen\np03['O2_corr'] = optode_O2_corr.O2_sal_comp(oxygen_conc=p03['02'],\n                                     salinity=p03.S,\n                                     temperature=p03.CT)\np03['O2_corr_umkg'] = optode_O2_corr.O2_molar2umkg(oxygen_conc=p03.O2_corr,\n                                     salinity=p03.S,\n                                     temperature=p03.CT,\n                                     pressure=p03.CD)\n\np03['O2PercentSat'] = optode_O2_corr.O2PercentSat(oxygen_conc=p03.O2_corr_umkg,\n                                     salinity=p03.S,\n              

In [19]:
#redeployment of instrument
p02.CD = p02.CD/100
p02.CT = p02.CT/1000
p02.CC = p02.CC/1000
p02.OT = p02.OT/1000
p02['02'] = p02['02']/100
p02.CH = (p02.CH-50)*0.0122
p02.TB = (p02.TB-50)*0.0247
# convert cond to salinity
p02['S'] = sw.salt(r=p02.CC/42.914,t=p02.CT,p=p02.CD)

### sal correct and convert oxygen
p02['O2_corr'] = optode_O2_corr.O2_sal_comp(oxygen_conc=p02['02'],
                                     salinity=p02.S,
                                     temperature=p02.CT)
p02['O2_corr_umkg'] = optode_O2_corr.O2_molar2umkg(oxygen_conc=p02.O2_corr,
                                     salinity=p02.S,
                                     temperature=p02.CT,
                                     pressure=p02.CD)

p02['O2PercentSat'] = optode_O2_corr.O2PercentSat(oxygen_conc=p02.O2_corr_umkg,
                                     salinity=p02.S,
                                     temperature=p02.CT,
                                     pressure=p02.CD)

p02['dep_lat']=56.873
p02['dep_lon']=-164.065

In [20]:
current_date = datetime.date.today().strftime('%Y%m%d')
"""p03.to_csv('19BSITAEPR2A_p03_20190'+month+'.'+day+'.csv', 
            date_format='%Y-%m-%dT%H:%M:%SZ')"""
p02.to_csv('19BSITAEPR2A_p02_20190'+month+'.'+day+'.csv', 
           date_format='%Y-%m-%d %H:%M:%S')
metdata.to_csv('19BSITAEPR2A_met_20190'+month+'.'+day+'.csv', 
               date_format='%Y-%m-%d %H:%M:%S')